# `ipytest` Summary

`ipytest` aims to make testing code in IPython notebooks easy. At its core, it offers a way to run pytest tests inside the notebook environment. It is also designed to make the transfer of the tests into proper python modules easy by supporting to use standard `pytest` features.

To get started install `ipytest` via:

```bash
pip install -U ipytest
```

To use `ipytest`, import it and configure the notebook. In most cases, running `ipytest.autoconfig()` will result in reasonable defaults:

- Tests can be executed with the `%%ipytest` magic
- The `pytest` assert rewriting system to get nice assert messages will integrated into the notebook 
- If not notebook name is given, a workaround using temporary files will be used

For more control, pass the relevant arguments to `ipytest.autconfig()`. For details, see the documentation in the readme.

In [1]:
import ipytest

ipytest.autoconfig()

## Execute tests

To execute test, just decorate the cells containing the tests with the `%%ipytest` magic:

In [2]:
%%ipytest

# define the tests

def test_my_func():
    assert my_func(0) == 0
    assert my_func(1) == 0
    assert my_func(2) == 2
    assert my_func(3) == 2
    
    
def my_func(x):
    return x // 2 * 2 

.                                                                                            [100%]
1 passed in 0.03s


## Using pytest fixtures

Common pytest features, such as fixtures and parametrize, are supported out of the box:

In [3]:
%%ipytest

import pytest

@pytest.mark.parametrize('input,expected', [
    (0, 0),
    (1, 0),
    (2, 2),
    (3, 2),
])
def test_parametrized(input, expected):
    assert my_func(input) == expected
    
    
@pytest.fixture
def my_fixture():
    return 42
    
    
def test_fixture(my_fixture):
    assert my_fixture == 42

.....                                                                                        [100%]
5 passed in 0.03s
